# Naive Bayes


<!-- PELICAN_BEGIN_SUMMARY -->

Niave Bayes is a classification technique that assumes strong independence between features. Even under these assumptions, it has been shown to perform well for certain tasks like spam filtering of text based messages. 

In this post, I will explain the theory behind Naive Bayes. 

We'll also take a look under the hood of Scikit-learn's implementation of Multinomail Bayes, and then we'll implement a stripped down version that retains the key features in order to understand what's going on.

<!-- PELICAN_END_SUMMARY -->


## Basic Theory

First, we need to define what question we are trying to answer. Let's stick to the SPAM/HAM filtering application. 

So given a message, what is the probability that is is HAM (normal message) or SPAM? 

The quantity we are after then is $p(S|D)$. What is the probability of SPAM (S) given a document (D). 

In the framework of Bayesian probability, this quantity is known as the posterior. Conveniently, Bayes Theorem defines a practical way to calculate this quantity:

$$ p(S|D) = \frac{P(D|S)*P(S)}{P(D)} = \frac{P(D,S)}{P(D)} $$

In words, this expression reads: the probability that a given document, D, belongs to SPAM is equal to the likelihood of observing the document (given that it belongs to SPAM) multiplied by the prior (the probability of drawing a SPAM document for a collection of messages) divided by the probability of observing the document (evidence)

In machine learning applications, the most convenient way to represent information about a particular sample is in a vector. In this case of message classification, we can vectorize the message based on word counts from a large vocabulary. In this way we transform our raw .txt data to a vector where each element denotes the number of times a word appears in the message. 

$$ f: D \rightarrow \vec{x} $$

Here $f$ is a function that vectorizes the input document D. The length of the resulting feature vector $\vec{x}$ corresponds to the number of unique words within a large number of sample documents. Again, the values of the vector entries $x_i$ represent the number of times the $i^{\text{th}}$ word appears in document D. 

There are more sophisticated ways to encode text into feature vectors, but for now we will stick the feature counts. 



Now that we have transformed our documents into feature vectors we rewrite the joint likelihood as 

\begin{align}
p(D,S) =& p(\vec{x},S)\\
       =& p(x_1 | x_2, x_3, ..., x_n, S)p(x_2, x_3, ..., x_n, S)\\
       &\vdots \\
       =& p(x_1| x_2, x_3, ..., x_n, S)p(x_2 | x_3, ..., x_n, S)... p(x_n | S)\\
\end{align}


Here, we make the assumption that the likelihood of a feature is independent of all other likelihoods:

$$ p(x_i | x_{i+1}, ..., x_n, S) = p(x_i | S) $$

so that 

$$ p(D,S) = p(\vec{x},S) = p(S) \prod_{i=1}^{n}p(x_i | S) $$

giving finally

\begin{align}
p(S | \vec{x})&=\frac{p(S) \prod_{i=1}^{n}p(x_i | S)}{p(\vec{x})}\\
&=\frac{p(S) \prod_{i=1}^{n}p(x_i | S)}{\sum_{k}p(S_k)p(\vec{x}|S_k)}\\
&=\frac{p(S) \prod_{i=1}^{n}p(x_i | S)}{\sum_{k}p(S_k)\prod_{i=1}^{n}p(x_i | S_k)}\\
\end{align}


In order to implement Naive Bayes, we now need to specify the form $p(\vec{x} | S)$ where the feature probabilities $p(x_i | S)$ are indpendent. There are several choices, all of which are implemented in sci-kit learn including Bernoulli, Gaussian, and Multinomial. 

In particular we will use the Multinomial Distribution: 
 
$$ p(\vec{x}|S_k) = \frac{(\sum_i x_i)!}{\prod_i x_i !}\prod_i p_{ki}^{xi}$$

The Multinomial distribution gives the probability of choosing the histogram defined by $\vec{x}$ for the class S.

Plugging this into the formula above and setting $\frac{(\sum_i x_i)!}{\prod_i x_i !} = a$ we get 

\begin{align}
p(S | \vec{x})
&=\frac{a p(S) \prod_{i=1}^{n}p_{ki}^{x_i}}{\sum_{k}p(S_k) a \prod_{i=1}^{n}p_{ki}^{x_i}}\\
&=\frac{p(S_k) \prod_{i=1}^{n}p_{ki}^{x_i}}{\sum_{k}p(S_k) \prod_{i=1}^{n}p_{ki}^{x_i}}\\
\end{align}

which means the prefactor $a$ cancels out! This ends up saving a lot of computational cost in the implementation.

By taking the log of the numerator above we arrive at an especially convenient form:

\begin{align}
\text{log}p(S | \vec{x})
&\propto \text{log} p(S) + \sum_{i=1}^{n} {x_i} \text{log}(p_{ki})\\
&\propto b + \vec{x_i}^{T}\cdot\vec{w}
\end{align}
 
 
which we'll call the joint log likelihood.

Hence to implement our model, we simply need to calculate the weight vectors $\vec{w}$ and the priors $p(S)$ for each class!


Now how do we practically calculate $p_{ki}$? This quantity just represents the probability that word $i$ occurs in class $S_k$, which can be estimated by the word counts from the training set:

$$ p_{ki} = \frac{ \sum_{t}x(t)_{ki} + \alpha}{\sum_i \sum_{t}{x(t)_{ki}} + N_{v} \alpha} $$

where the sum over $t$ represents a sum over all of the training examples in that class $k$. In words, this expression is the number of instances of word $i$ within the class $k$ divided by the total number of all word counts in class $k$. 

If $p_{ki}$ is zero, we will run into problems since any time $p_{ki}=0$ it will destroy all other information when being multiplied by other $p_{ki}$. Thus, we insert $\alpha$ which is known as Laplace smoothing. 









## Implementation

Training Stage: 

Inputs: Training Set consisting of $n$ messages which have been converted to $1\times m$ vectors $\vec{x}$ along with labels of length $n$, $\vec{y}$. The Data matrix, $X$ is then composed as $n$ row vecotors of length $m$ giving a $n\times m$ matrix. 

Result: Parameterized model: i.e. all of the $p_{ki}$ and $P(S_k)$ for each word in each class.

Prediction Stage:

Input: Test data arranged as $n\times m$ matrix X.

Output: Probabilities that each input message belongs to each class, in a $n \times \#\text{classes}$ matrix $Y$. 



### Attributes of class test_MultinomialNB()

##### self.W

These are the 'weights' for all classes, $k$:

\begin{align}
W &= \begin{bmatrix} \text{Log}(\vec{p_{0}}) \\
                     \text{Log}(\vec{p_{1}})
                     \end{bmatrix}\\
 &=\begin{bmatrix} \text{Log}(\frac{ \sum_{t}x(t)_{ki} + \alpha}{\sum_i \sum_{t}{x(t)_{ki}} + N_{v} \alpha}) \\
                     \text{Log}(\frac{ \sum_{t}x(t)_{ki} + \alpha}{\sum_i \sum_{t}{x(t)_{ki}} + N_{v} \alpha})
                     \end{bmatrix}\\
&=\text{log}\begin{bmatrix} (\frac{Y^T X^T + \alpha }{\sum_i Y^T X^T + \alpha})
                     \end{bmatrix}\\
\end{align}

where $p_0$ represents the feature counts for HAM, and $p_1$ is SPAM, for example.


In [72]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import check_array
from sklearn.utils.extmath import safe_sparse_dot
import numpy as np



class test_MultinomialNB():
    def __init__(self, alpha=1.0):
        self.alpha  = alpha
    """    
     We wish to predict the probability that a document
     described by feature vector x [1,n_features] belongs
     to class C_k. i.e. P(C_k | x)
     
     Using Bayes Theroem, we can compute the posterior (above) 
     from the likelihood*prior i.e.
     
     P( C_k | x ) = P( x | C_k ) * P( C_k ) / P( x )
     
     or taking the Log:
     
     Log[ P( C_k | x )] = Log[ P( x | C_k ) ] + Log[ P( C_k ) ] - Log[ P( x ) ]
                     = Log[ P( x , C_k ) ] - Log[ Z ]
                     = Log[ JL ] - Log[ Z ]
                     
    where JL means Joint Likelihood.
    
    JL = P(x, C_k) = P(x1 | x2, x3, ... , C_k)*P(x2 | x3, x4, ... , C_k) * ... * P(xn | C_k) * P(C_k)
    
    Under the assumption of independent features the conditional probabilities simplify to:
    
    JL = P(C_k) * prod_i P(x_i | C_k)
    
    if we assume a multinomial distribution for P(x | C_k) then we have 
    
    JL ~ P(C_k) * prod_i (p_ki ^ x_i)
    
    or taking Log
    
    JLL ~ Log[ P(C_k) ] + sum_i (x_i * Log[ p_ki ])
    
    which in vector form is 
    
    JLL ~ b + dot(x,w_k)
    
        where w = Log[ p_k ]
        
    
    Z is the 'evidence' which can be computed
        
    Z = sum_k P( C_k ) * P(x | C_k)
      = sum_k JL
    which is just the sum of the Joint Likelihood across all classes.
    
    *** 
    To fit the model then, we need to compute the matrix 'W' of shape [n_classes, n_features]
    where 
        W_ki = Log [ p_ki ]
             = Log [ (sum_{documents in class k} x_i + alpha ) / (sum_{all documents} + n*alpha ]
    
    We also need the vector 'b'[n_classes,1] which will be a vector of the priors Log[ C_k]
    where 
        b_k = Log[ num_documents in class k / total_num_documents ]
    
    
    """
    def fit(self, X, y):
        # Tranform y [n_samples,1] where y_d = label
        # to Y [n_samples,n_classes] where  y_dk = 1 if doc d belongs to class k
        # else y_dk = 0.
        labelbin = LabelBinarizer()
        Y = labelbin.fit_transform(y)
        self.classes_ = labelbin.classes_
        Y = Y.astype(np.float64) # cast as floats
        if Y.shape[1] == 1:
            Y = np.concatenate((1 - Y, Y), axis=1)


        X = check_array(X, accept_sparse='csr', dtype=np.float64)
        # Now with Y, feature counts for each class can be computed using 
        # matrix math i.e.
        # fc = Y.T * X
        # [n_classes, n_features] = [n_classes, n_samples] * [n_samples, n_features]

        feature_counts = safe_sparse_dot(Y.T, X)
        
        
        # Now smooth the feature counts since any zero instance will kill all other non-zero proba's
        smooth_fc = feature_counts + self.alpha

        # Compute matrix W as Log [ smooth_fc ] - Log [ smooth_fc.sum(axis=1)]


        self.W = np.log(smooth_fc) - np.log(smooth_fc.sum(axis=1).reshape(-1,1)) 
        # Compute vector 'b' as b_k = Log[ n_samples in class k ] - Log[ n_samples ]
        self.b = np.log(Y.sum(axis=0)) - np.log(Y.shape[0]) 

        
        
    def _JLL(self,X):
        # Compute Joint Log Likelihood = b*np.ones((b.shape[0],X)) + W * X.T 
        # shape _JLL = [ n_classes, n_samples ]
        return self.b.reshape(self.b.shape[0],-1) + safe_sparse_dot(self.W,X.T)
    def predict(self,X):
        return self.classes_[np.argmax(self._JLL(X),axis=0)]
    def predict_log_proba(self,X):
        log_Z = logsumexp(self._JLL(X),axis=1)
        return self._JLL(X) - log_Z #shape?
    def predict_proba(self,X):
        return np.exp(self.predict_log_proba(X))
    
        
        
        

Testing...

In [6]:
import numpy as np
from sklearn.metrics import classification_report,\
    f1_score, accuracy_score, confusion_matrix
data = np.random.randint(10,size=(20,10))
labels = np.random.randint(2,size=20)
mnb_classifier = test_MultinomialNB()
mnb_classifier.fit(data,labels)
predictions = mnb_classifier.predict(data)
predictions - labels

print classification_report(labels,predictions)
print confusion_matrix(labels,predictions)

SyntaxError: invalid syntax (<ipython-input-6-4c26415873f4>, line 10)

In [76]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
data = np.random.randint(10,size=(20,50))
labels = np.random.randint(2,size=20)
mnb_classifier = MultinomialNB()
mnb_classifier.fit(data,labels)
mnb_classifier.predict(data) - labels

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

the equation for proba is $x = \sum_i x_i$

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:50% !important; }</style>"))

In [ ]:
for feature in raw_data['feature_names']:
    facet = sns.FacetGrid(data=data,hue='class')
    facet.map(sns.kdeplot,feature)

In [ ]:
sns.boxplot(data=data,x=data.target,y=data.alcohol)
predictions = wine_detector.predict(data_test)
print('accuracy', accuracy_score(label_test, predictions))
print('confusion matrix\n', 
      confusion_matrix(label_test, predictions))
print('(row=expected, col=predicted)')